# Developing behavioral data preprocessing pipeline for OPM MEG analysis


## Preprocessing Steps incuded:
- Loading, syncing, and alligning data 
- Categorizing ACC and EMG data into rest vs muscular activity vs movement
-


### 0. Importing

In [ ]:
# general packages
import json
import os
import importlib
import sys
import numpy as np
import pandas as pd
import pyxdf
from itertools import compress
import matplotlib.pyplot as plt
import datetime as dt

import mne

# ephys packages
# from mne.filter import filter_data, notch_filter

In [ ]:
def add_repo_dir():
    """adds local repo directory to sys to allow importing from repo"""

    wd = os.getcwd()

    COUNTER = 20  #  to prevent eternal while loop

    while not wd.endswith('lid_opm'):
        wd = os.path.dirname(wd)
        COUNTER -= 1

        if COUNTER == 0:
            raise ValueError('repo dir not found!')

    print(f'add repo directory to sys: {wd} ')

    sys.path.append(wd)

    return

In [ ]:
# add custom functions

add_repo_dir()

import utils.load_utils as load_utils
from source_raw_conversion import load_source_opm as source_opm


## 1. Load behavioral source data

Define:
- subject
- task
- configuration version


In [ ]:
CONFIG_VERSION = "v1"

SUB = '03'  # 
TASK = 'rest'
ACQ = 'predopa'


# load settings
sub_config = load_utils.load_subject_config(subject_id=SUB,)
preproc_config = load_utils.load_preproc_config(version=CONFIG_VERSION,)
sub_meta_info = load_utils.get_sub_rec_metainfo(config_sub=sub_config)




In [ ]:
# sub_meta_info

In [ ]:
import source_raw_conversion.load_lsl as loadlsl
import source_raw_conversion.time_syncing as sync
import source_raw_conversion.load_source_opm as sourceopm

import signal_processing.preprocessing as preproc
import plotting.processing_checks as proc_plotting

manual dev of source lsl to raw flow

In [ ]:
importlib.reload(proc_plotting)
importlib.reload(preproc)
importlib.reload(loadlsl)
importlib.reload(sync)

# deprecated flow
# (
#     auxdat, aux_chnames,
#     aux_sfreq, pyg_timings
# ) = loadlsl.convert_source_lsl_to_raw(SUB, TASK, ACQ)

for REC in sub_meta_info['rec_name']:
    print(REC)

    try:
        TASK, ACQ = REC.split('_')
    except:
        print(f'\n##### WARNING: {REC} skipped\n')
        continue
    
    recRaw = preproc.rawData_singleRec(SUB, TASK, ACQ)

    if recRaw.task == 'rest': continue 

    proc_plotting.plot_emgacc_check_for_tasks(recRaw, SAVE=False, SHOW=False,)
    # include sample sizes in plt


In [ ]:
preproc_config

In [ ]:
importlib.reload(proc_plotting)
importlib.reload(preproc)
importlib.reload(loadlsl)
importlib.reload(sync)
importlib.reload(source_opm)

recRaw = preproc.rawData_singleRec(
    SUB, TASK, ACQ, INCL_OPM=True,
    OPM_PREPROC={
        'resample': True, 'bandpass': True,
        'notch': True, 'hfc': False
    }
)


check hfc projections, for now too little sensors probably, therefore no conversion of SVD math

In [ ]:
### HFC CHECK
# hfc_projs = recRaw.OPM_Z.info['projs']

# for p in hfc_projs:
#     # print('Name:', p['desc'], 'Active:', p.get('active', False))
#     cols = p.get('data', {}).get('col_names', None)
#     # print('  cols:', cols)


# # print(recRaw.OPM_Z.ch_names)
# # print(recRaw.OPM_Z.info['bads'])

# print(hfc_projs[0]['data']['col_names']) 
# print(hfc_projs[0]['data']['data'])          # should be your MEG channel names
# print(np.linalg.norm(hfc_projs[0]['data']['data']))  # should NOT be 0

# # check sensor geometry, should not be close to 0
# pos = np.array([ch['loc'][:3] for ch in recRaw.OPM_Z.info['chs']])
# print("Sensor bounds (min, max) in meters:\n", pos.min(axis=0), pos.max(axis=0))

Check orientation of first 3 and last 3 orientation normvector rotations

In [ ]:
# run for 3d turning plot
%matplotlib notebook

### orientation solving

chnames = recRaw.OPM_Z.ch_names

# chname = '15Z'
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for chname in chnames:
    ch = recRaw.OPM_Z.info['chs'][chnames.index(chname)]
    # print(ch['loc'])
    pos = ch['loc'][:3]            # channel position in meters
    ori_z = ch['loc'][3:6]         # first orientation vector
    ori_y = ch['loc'][6:9]         # second orientation vector (if available)

    # Plot sensor location
    ax.scatter(*pos, c='k', s=20,)

    # Plot orientation vectors (scaled for visibility)
    scale = .005
    ax.quiver(*pos, *ori_z, length=scale, color='r',)
    ax.quiver(*pos, *ori_y, length=scale, color='b',)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.legend()
ax.set_title(f"Orientation vectors ")
plt.show()

In [ ]:
# set interactive plotting back
%matplotlib inline

preprocess opms

In [ ]:
recRaw.OPM_Z.info

In [ ]:
opm_epochs = mne.Epochs(
    raw=recRaw.OPM_Z, events=recRaw.event_arr,
    event_id=recRaw.event_codes,
    tmin=0, tmax=3, baseline=None, preload=True,
    reject=None,
)

### check (ICA) cleaning before epoching
# TODO select on event type, test topograms per event type

In [ ]:
opm_epochs.get_data().shape

In [ ]:
STIM = 'rest'  # 'go_left

ep_i = 5

chs_e = opm_epochs[STIM].get_data()[ep_i, :, :]
fs = recRaw.OPM_Z.info['sfreq']

for i, ch in enumerate(chs_e):
    plt.plot(np.arange(len(ch)) / fs, ch,
             label=recRaw.OPM_Z.info['ch_names'][i])

    if i > 20: break

plt.legend()

plt.show()



In [ ]:
opm_epochs[STIM].compute_psd().plot()

plt.show()

In [ ]:
psds, freqs = mne.time_frequency.psd_array_welch(
    opm_epochs[STIM].get_data(),
    fmin=13, fmax=20,
    n_fft=int(recRaw.OPM_Z.info['sfreq']),
    sfreq=recRaw.OPM_Z.info['sfreq'],
)



In [ ]:

psds_plot = psds.mean(axis=(0, 2))  # gives mean PSD-power within defined range per channel

mne.viz.plot_topomap(
    psds_plot,
    opm_epochs[STIM].info,
    cmap="viridis",  # for diff "RdBu_r"
    sensors=True,        # show sensor dots
    outlines="head",     # no change for meg, should add head circle, ears, nose
    contours=1,
)

plt.show()


extended cleaning


test further cleaning, HFC does not converge, try ICA for specific (stationary) artefacts

In [ ]:
recRaw.OPM_Z.get_data().shape

### Explore visualization

- calculate spectral envelops (analytical signals) for theta, alpha, beta, gamma
- plot envelops over 3 second epoch windows, average over channels and over epochs, resulting in mean envelop over the course of specific task
- compare contra-lateral vs ipsi-lateral hemisphere

- plot next to ACC-hand, and mean-envelop from EMG per extremity (deltoid + brachrad)